In [1]:
import matplotlib as mpl
import matplotlib.pylab as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
import numpy as np
import scipy as sp
import pandas as pd
import statsmodels.api as sm
import sklearn as sk

## 1. 데이터 불러오기

In [2]:
from sklearn.datasets import load_boston

boston = load_boston()

In [3]:
dfX = pd.DataFrame(boston.data, columns=boston.feature_names)
dfy = pd.DataFrame(boston.target, columns=["MEDV"])

In [4]:
df = pd.concat([dfX, dfy], axis=1)
df.tail()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
501,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1.0,273.0,21.0,391.99,9.67,22.4
502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1.0,273.0,21.0,396.90,9.08,20.6
503,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1.0,273.0,21.0,396.90,5.64,23.9
504,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1.0,273.0,21.0,393.45,6.48,22.0
505,0.04741,0.0,11.93,0.0,0.573,6.030,80.8,2.5050,1.0,273.0,21.0,396.90,7.88,11.9


In [5]:
df.shape

(506, 14)

## 2. 범주형 데이터 더미변수화

In [6]:
df.dtypes

CRIM       float64
ZN         float64
INDUS      float64
CHAS       float64
NOX        float64
RM         float64
AGE        float64
DIS        float64
RAD        float64
TAX        float64
PTRATIO    float64
B          float64
LSTAT      float64
MEDV       float64
dtype: object

In [7]:
df["ZN"].unique()

array([ 18. ,   0. ,  12.5,  75. ,  21. ,  90. ,  85. , 100. ,  25. ,
        17.5,  80. ,  28. ,  45. ,  60. ,  95. ,  82.5,  30. ,  22. ,
        20. ,  40. ,  55. ,  52.5,  70. ,  34. ,  33. ,  35. ])

In [8]:
df["CHAS"].unique()

array([0., 1.])

In [9]:
df["RAD"].unique()

array([ 1.,  2.,  3.,  5.,  4.,  8.,  6.,  7., 24.])

In [10]:
df["TAX"].unique()

array([296., 242., 222., 311., 307., 279., 252., 233., 243., 469., 226.,
       313., 256., 284., 216., 337., 345., 305., 398., 281., 247., 270.,
       276., 384., 432., 188., 437., 403., 193., 265., 255., 329., 402.,
       348., 224., 277., 300., 330., 315., 244., 264., 223., 254., 198.,
       285., 241., 293., 245., 289., 358., 304., 287., 430., 422., 370.,
       352., 351., 280., 335., 411., 187., 334., 666., 711., 391., 273.])

"CHAS" 컬럼의 값으로 0과 1만 존재하므로, 범주형 데이터임을 알 수 있습니다.  
"CHAS" 컬럼을 더미변수화합니다.

In [11]:
df["CHAS"] = df["CHAS"].astype("int32") # int형으로 형변환

In [12]:
df["CHAS"].value_counts()

0    471
1     35
Name: CHAS, dtype: int64

In [13]:
from patsy import dmatrix

dmatrix("C(CHAS) + 0", df) 

DesignMatrix with shape (506, 2)
  C(CHAS)[0]  C(CHAS)[1]
           1           0
           1           0
           1           0
           1           0
           1           0
           1           0
           1           0
           1           0
           1           0
           1           0
           1           0
           1           0
           1           0
           1           0
           1           0
           1           0
           1           0
           1           0
           1           0
           1           0
           1           0
           1           0
           1           0
           1           0
           1           0
           1           0
           1           0
           1           0
           1           0
           1           0
  [476 rows omitted]
  Terms:
    'C(CHAS)' (columns 0:2)
  (to view full data, use np.asarray(this_obj))

In [14]:
dummy_data = np.asarray(dmatrix("C(CHAS) + 0", df))
dummy_data

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]])

In [15]:
dummy_df = pd.DataFrame(dummy_data, columns=["CHAS[0]", "CHAS[1]"])
dummy_df

,CHAS[0],CHAS[1]
0,1.0,0.0
1,1.0,0.0
2,1.0,0.0
3,1.0,0.0
4,1.0,0.0
...,...,...
501,1.0,0.0
502,1.0,0.0
503,1.0,0.0
504,1.0,0.0


In [16]:
# 더미변수화한 변수를 기존 데이터프레임에 추가
# 더미변수화되기 전의 컬럼을 삭제
df = pd.concat([df, dummy_df], axis=1)
df = df.drop("CHAS", axis=1)
df

,CRIM,ZN,INDUS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV,CHAS[0],CHAS[1]
0,0.00632,18.0,2.31,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0,1.0,0.0
1,0.02731,0.0,7.07,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6,1.0,0.0
2,0.02729,0.0,7.07,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7,1.0,0.0
3,0.03237,0.0,2.18,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4,1.0,0.0
4,0.06905,0.0,2.18,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0.573,6.593,69.1,2.4786,1.0,273.0,21.0,391.99,9.67,22.4,1.0,0.0
502,0.04527,0.0,11.93,0.573,6.120,76.7,2.2875,1.0,273.0,21.0,396.90,9.08,20.6,1.0,0.0
503,0.06076,0.0,11.93,0.573,6.976,91.0,2.1675,1.0,273.0,21.0,396.90,5.64,23.9,1.0,0.0
504,0.10959,0.0,11.93,0.573,6.794,89.3,2.3889,1.0,273.0,21.0,393.45,6.48,22.0,1.0,0.0


## 3. 결측 데이터 확인

In [17]:
import missingno as msno

msno.matrix(df)
plt.show()

In [18]:
msno.bar(df)
plt.show()

모든 컬럼에 결측치가 없음을 알 수 있습니다.

## 4. 데이터 스케일링

In [19]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

In [20]:
column_list = [column for column in df.columns if column not in ["CHAS[0]", "CHAS[1]"]]
column_list

['CRIM',
 'ZN',
 'INDUS',
 'NOX',
 'RM',
 'AGE',
 'DIS',
 'RAD',
 'TAX',
 'PTRATIO',
 'B',
 'LSTAT',
 'MEDV']

In [21]:
scaler.fit(df[column_list])

StandardScaler(copy=True, with_mean=True, with_std=True)

In [22]:
scaler.transform(df[column_list])


array([[-0.41978194,  0.28482986, -1.2879095 , ...,  0.44105193,
        -1.0755623 ,  0.15968566],
       [-0.41733926, -0.48772236, -0.59338101, ...,  0.44105193,
        -0.49243937, -0.10152429],
       [-0.41734159, -0.48772236, -0.59338101, ...,  0.39642699,
        -1.2087274 ,  1.32424667],
       ...,
       [-0.41344658, -0.48772236,  0.11573841, ...,  0.44105193,
        -0.98304761,  0.14880191],
       [-0.40776407, -0.48772236,  0.11573841, ...,  0.4032249 ,
        -0.86530163, -0.0579893 ],
       [-0.41500016, -0.48772236,  0.11573841, ...,  0.44105193,
        -0.66905833, -1.15724782]])

In [23]:
df[column_list] = scaler.transform(df[column_list])
df.tail()

,CRIM,ZN,INDUS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV,CHAS[0],CHAS[1]
501,-0.413229,-0.487722,0.115738,0.158124,0.439316,0.018673,-0.625796,-0.982843,-0.803212,1.176466,0.387217,-0.418147,-0.014454,1.0,0.0
502,-0.415249,-0.487722,0.115738,0.158124,-0.234548,0.288933,-0.716639,-0.982843,-0.803212,1.176466,0.441052,-0.500850,-0.210362,1.0,0.0
503,-0.413447,-0.487722,0.115738,0.158124,0.984960,0.797449,-0.773684,-0.982843,-0.803212,1.176466,0.441052,-0.983048,0.148802,1.0,0.0
504,-0.407764,-0.487722,0.115738,0.158124,0.725672,0.736996,-0.668437,-0.982843,-0.803212,1.176466,0.403225,-0.865302,-0.057989,1.0,0.0
505,-0.415000,-0.487722,0.115738,0.158124,-0.362767,0.434732,-0.613246,-0.982843,-0.803212,1.176466,0.441052,-0.669058,-1.157248,1.0,0.0
